In [2]:
# Setup Keys
import os
from dotenv import load_dotenv
load_dotenv()


# Tactic 1 : Use delimiters or pointer

In [4]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Summarize the text that part of the Content only into single sentence.
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          Background :
          You should express what you want a model to do by
          providing instructions that are as clear and
          specific as you can possibly make them.
          This will guide the model towards the desired output,
          and reduce the chances of receiving irrelevant
          or incorrect responses. Don't confuse writing a
          clear prompt with writing a short prompt.
          In many cases, longer prompts provide more clarity
          and context for the model, which can lead to
          more detailed and relevant outputs.

          Content :
          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the NBA Most Valuable Player (MVP) for the 2024–25 NBA season. During that season, Gilgeous-Alexander also led the Thunder to their first NBA championship since relocating from Seattle to Oklahoma City, becoming the 11th Canadian to win an NBA title.

          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': "\n          Background : \n          You should express what you want a model to do by\n          providing instructions that are as clear and\n          specific as you can possibly make them.\n          This will guide the model towards the desired output,\n          and reduce the chances of receiving irrelevant\n          or incorrect responses. Don't confuse writing a\n          clear prompt with writing a short prompt.\n          In many cases, longer prompts provide more clarity\n          and context for the model, which can lead to\n          more detailed and relevant outputs.\n\n          Content : \n          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the 

In [6]:
response

{'messages': [HumanMessage(content=[{'type': 'text', 'text': "\n          Background : \n          You should express what you want a model to do by\n          providing instructions that are as clear and\n          specific as you can possibly make them.\n          This will guide the model towards the desired output,\n          and reduce the chances of receiving irrelevant\n          or incorrect responses. Don't confuse writing a\n          clear prompt with writing a short prompt.\n          In many cases, longer prompts provide more clarity\n          and context for the model, which can lead to\n          more detailed and relevant outputs.\n\n          Content : \n          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team 

# Tactic 2 : Use structured output

In [11]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Summarize the text that part of the Content only with this format :
    1. Origin Story
    2. Draft Year
    3. Draft Team
    4. Current Team
    5. Current Team Tenure
    6. NBA Achievement
    7. National Team Achievement
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          Background :
          You should express what you want a model to do by
          providing instructions that are as clear and
          specific as you can possibly make them.
          This will guide the model towards the desired output,
          and reduce the chances of receiving irrelevant
          or incorrect responses. Don't confuse writing a
          clear prompt with writing a short prompt.
          In many cases, longer prompts provide more clarity
          and context for the model, which can lead to
          more detailed and relevant outputs.

          Content :
          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the NBA Most Valuable Player (MVP) for the 2024–25 NBA season. During that season, Gilgeous-Alexander also led the Thunder to their first NBA championship since relocating from Seattle to Oklahoma City, becoming the 11th Canadian to win an NBA title.[2]

          Gilgeous-Alexander played one season of college basketball for the Kentucky Wildcats. He was selected 11th overall by the Charlotte Hornets in the first round of the 2018 NBA draft before being traded to the Los Angeles Clippers on draft night. In his rookie season with the Clippers, Gilgeous-Alexander made an appearance in the 2019 NBA playoffs and was named to the NBA All-Rookie Second Team. Following his rookie season, he was traded to the Oklahoma City Thunder in July 2019.

          In his first season with the Thunder, Gilgeous-Alexander was their leading scorer and led the team to the 2020 NBA playoffs as the fifth seed. After dealing with injuries over the next two seasons, he was named to his first NBA All-Star Game and was voted to the All-NBA First Team in 2023, finishing fourth in the league in scoring with 31.4 points per game. In the 2024–25 NBA season, Gilgeous-Alexander won the NBA Most Valuable Player (MVP) award after leading the league in scoring with an average of 32.7 points per game. He later won the NBA Finals MVP award as the Thunder won the 2025 NBA Finals, becoming the fourth player in NBA history to win the MVP award, Finals MVP award, and scoring title all in the same season. Additionally, Gilgeous-Alexander became the second Canadian to win the MVP award, following Steve Nash, who first claimed it in back-to-back seasons in 2005 and 2006, and the first Canadian ever to win the Finals MVP award.[3][4]

          With the Canada men's national basketball team, Gilgeous-Alexander won the bronze medal at the 2023 FIBA Basketball World Cup and was named to the All-Tournament Team. In 2023, he received the Northern Star Award as Canada's Athlete of the Year, becoming only the second basketball player to earn the award since Steve Nash first won it in 2005.

          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': "\n          Background : \n          You should express what you want a model to do by\n          providing instructions that are as clear and\n          specific as you can possibly make them.\n          This will guide the model towards the desired output,\n          and reduce the chances of receiving irrelevant\n          or incorrect responses. Don't confuse writing a\n          clear prompt with writing a short prompt.\n          In many cases, longer prompts provide more clarity\n          and context for the model, which can lead to\n          more detailed and relevant outputs.\n\n          Content : \n          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the 

# Tactic 3 : Use certain criteria that the input need to be satisfied

In [19]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Defined Content as the section after word "Content :",  so first return the number of Content section detected

    Summarize the text that part of the Content only with this format :
    1. Origin Story
    2. Draft Year
    3. Draft Team
    4. Current Team
    5. Current Team Tenure
    6. NBA Achievement
    7. National Team Achievement

    If there are multiple Content section listed it down with numerical order with the Title that best describe it

    If the you feel like the content information can't be written on the given format just return 'Content Does Not Match Format'
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          Content :
          You should express what you want a model to do by
          providing instructions that are as clear and
          specific as you can possibly make them.
          This will guide the model towards the desired output,
          and reduce the chances of receiving irrelevant
          or incorrect responses. Don't confuse writing a
          clear prompt with writing a short prompt.
          In many cases, longer prompts provide more clarity
          and context for the model, which can lead to
          more detailed and relevant outputs.

          Content :
          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the NBA Most Valuable Player (MVP) for the 2024–25 NBA season. During that season, Gilgeous-Alexander also led the Thunder to their first NBA championship since relocating from Seattle to Oklahoma City, becoming the 11th Canadian to win an NBA title.[2]

          Gilgeous-Alexander played one season of college basketball for the Kentucky Wildcats. He was selected 11th overall by the Charlotte Hornets in the first round of the 2018 NBA draft before being traded to the Los Angeles Clippers on draft night. In his rookie season with the Clippers, Gilgeous-Alexander made an appearance in the 2019 NBA playoffs and was named to the NBA All-Rookie Second Team. Following his rookie season, he was traded to the Oklahoma City Thunder in July 2019.

          In his first season with the Thunder, Gilgeous-Alexander was their leading scorer and led the team to the 2020 NBA playoffs as the fifth seed. After dealing with injuries over the next two seasons, he was named to his first NBA All-Star Game and was voted to the All-NBA First Team in 2023, finishing fourth in the league in scoring with 31.4 points per game. In the 2024–25 NBA season, Gilgeous-Alexander won the NBA Most Valuable Player (MVP) award after leading the league in scoring with an average of 32.7 points per game. He later won the NBA Finals MVP award as the Thunder won the 2025 NBA Finals, becoming the fourth player in NBA history to win the MVP award, Finals MVP award, and scoring title all in the same season. Additionally, Gilgeous-Alexander became the second Canadian to win the MVP award, following Steve Nash, who first claimed it in back-to-back seasons in 2005 and 2006, and the first Canadian ever to win the Finals MVP award.[3][4]

          With the Canada men's national basketball team, Gilgeous-Alexander won the bronze medal at the 2023 FIBA Basketball World Cup and was named to the All-Tournament Team. In 2023, he received the Northern Star Award as Canada's Athlete of the Year, becoming only the second basketball player to earn the award since Steve Nash first won it in 2005.


          Content :
          Dixon was born on October 28, 1995, in Annapolis, Maryland. He attended Virginia Commonwealth University in Richmond, Virginia. Around this time, he released two studio albums, his debut Who Taught You to Hate Yourself? in 2016 and his second album, The Importance of Self Belief.[citation needed]

On May 7, 2021, Dixon released his third studio album, For My Mama and Anyone Who Look Like Her, which was met with critical acclaim. Lucy Shanker from Consequence gave the album an "A" rating calling the album "a work of art. It encompasses numerous genres; it's a jazz piece, it’s a marching band showcase, it's a spoken word performance, it's a rap album. It's a stunning exploration of the Black experience."[1] Kyle Kohner, writing for Exclaim! said the album is "compellingly verbose and, at times, boastful rapping style, but he also wields an impressive level of lyrical complexity. The emcee operates with a film director's eye, one that places himself in the actor's shoes, often offering multi-pronged characters that aim to make sense of himself and those who look like him."[2] On Beats Per Minute, Conor Lochrie described the album "an unfolding narrative which could be considered a thesis, Dixon emphasizes the commodification of Black art by White people: where Black art often seems like it has to have a greater point, a higher purpose – indulging in traumatic images of violence in racial horror, as in the above case – white people, contrastingly, get to make whatever art they wish."[3]

In 2022, Dixon appeared in Soul Glo's track "Spiritual Level of Gang Shit" on their studio album Diaspora Problems. On March 7, 2023, Dixon announced the album Beloved! Paradise! Jazz!? and released the single "Run, Run, Run". The full album was released through City Slang on June 2, 2023, and attracted widespread critical acclaim.[4] Robin Murray of Clash noted that the album title is derived "from a phrase used by the novelist Toni Morrison", and identified the choice of title as a nod to the "literary flair" of Dixon's output.[5]

On February 19, 2025, Dixon announced his fifth studio album, Magic, Alive!, which was released by City Slang on June 6. With the announcement came the lead single, "Sugar Water", featuring Quelle Chris and Anjimile.[6]

          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': '\n          Content : \n          You should express what you want a model to do by\n          providing instructions that are as clear and\n          specific as you can possibly make them.\n          This will guide the model towards the desired output,\n          and reduce the chances of receiving irrelevant\n          or incorrect responses. Don\'t confuse writing a\n          clear prompt with writing a short prompt.\n          In many cases, longer prompts provide more clarity\n          and context for the model, which can lead to\n          more detailed and relevant outputs.\n\n          Content : \n          Shaivonte Aician Gilgeous-Alexander (/ˈʃeɪ ˈɡɪldʒəs/ SHAY GHIL-jəs;[1] born July 12, 1998), also known by his initials SGA, is a Canadian professional basketball player for the Oklahoma City Thunder of the National Basketball Association (NBA). He is a four-time NBA All-Star, a three-time All-NBA First Team member, and was named the NB

# Tactic 4 : Give snapshot of example

In [23]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Your task is to answer in a consistent style based on these example :

    <child>: Teach me about patience.

    <grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note the most intricate tapestry begins with a solitary thread.
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          <child>: Teach me about resilience.
          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': '\n          <child>: Teach me about resilience.\n          '}]
Answer : <grandparent>: The mightiest oak was once a small acorn that refused to break beneath the weight of winter snow. The mountain stream does not rage against the stone—it simply flows around, over, and through, until even the hardest rock yields to its gentle persistence. So too must you bend without breaking, rest without surrendering, and return again and again to what matters, knowing that each stumble teaches your feet where to find firmer ground.


# Tactic 5 : Give specific steps and prompt the model to work out all of it first

In [25]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Your task is to determine if the student's solution is correct or not.

    To solve the problem do the following:
    - First, work out your own solution to the problem including the final total.
    - Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.
    Don't decide if the student's solution is correct until
    you have done the problem yourself.

    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as actual solution just calculated:
    ```
    yes or no
    ```
    Student grade:
    ```
    correct or incorrect
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          Question:
          ```
          I'm building a solar power installation and I need help working out the financials.
          - Land costs $100 / square foot
          - I can buy solar panels for $250 / square foot
          - I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
          What is the total cost for the first year of operations as a function of the number of square feet.
          ```
          Student's solution:
          ```
          Let x be the size of the installation in square feet.
          Costs:
          1. Land cost: 100x
          2. Solar panel cost: 250x
          3. Maintenance cost: 100,000 + 100x
          Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
          ```

          Actual solution:
          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
print(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': "\n          Question:\n          ```\n          I'm building a solar power installation and I need help working out the financials. \n          - Land costs $100 / square foot\n          - I can buy solar panels for $250 / square foot\n          - I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot\n          What is the total cost for the first year of operations as a function of the number of square feet.\n          ``` \n          Student's solution:\n          ```\n          Let x be the size of the installation in square feet.\n          Costs:\n          1. Land cost: 100x\n          2. Solar panel cost: 250x\n          3. Maintenance cost: 100,000 + 100x\n          Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000\n          ```\n          \n          Actual solution:\n          "}]
Answer : Question:
```
I'm building a solar power installation and I need help working out

# Tactic 6 : Inferring Topic

In [35]:
# Import package
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Determine whether each item in the following list of topics is a topic in the text below, which is delimited with triple backticks.
    Give your answer in JSON format with the topic as the keys and 1/0 where 1 determine that the text contains the topic

    After that return the langunage used in the text as the values for keys "original_languange" in the JSON output

    After that return the translated version of the text to the languange specified by the user and put it as the values for key "translated" in the JSON output, and also put the languange input by the user as the value for keys "translated_languange"

    So the output format will be
    {
      '<topic 1>': ,
      '<topic 2>': ,
      .... ,
      'original_languange': ,
      'translated_languange': ,
      'translated': ,

    }
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

story = """
  In a recent survey conducted by the government,
  public sector employees were asked to rate their level
  of satisfaction with the department they work at.
  The results revealed that NASA was the most popular
  department with a satisfaction rating of 95%.

  One NASA employee, John Smith, commented on the findings,
  stating, "I'm not surprised that NASA came out on top.
  It's a great place to work with amazing people and
  incredible opportunities. I'm proud to be a part of
  such an innovative organization."

  The results were also welcomed by NASA's management team,
  with Director Tom Johnson stating, "We are thrilled to
  hear that our employees are satisfied with their work at NASA.
  We have a talented and dedicated team who work tirelessly
  to achieve our goals, and it's fantastic to see that their
  hard work is paying off."

  The survey also revealed that the
  Social Security Administration had the lowest satisfaction
  rating, with only 45% of employees indicating they were
  satisfied with their job. The government has pledged to
  address the concerns raised by employees in the survey and
  work towards improving job satisfaction across all departments.
"""

message = HumanMessage(
    content=[
      {'type':'text',
       'text':f"""
          List of topics : {topic_list}

          Content text : {story}

          Please also translated it into Thai languange
          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
pprint(f'Answer : {response['messages'][-1].content}')

Prompt : [{'type': 'text', 'text': '\n          List of topics : [\'nasa\', \'local government\', \'engineering\', \'employee satisfaction\', \'federal government\']\n       \n          Content text : \n  In a recent survey conducted by the government, \n  public sector employees were asked to rate their level \n  of satisfaction with the department they work at. \n  The results revealed that NASA was the most popular \n  department with a satisfaction rating of 95%.\n\n  One NASA employee, John Smith, commented on the findings, \n  stating, "I\'m not surprised that NASA came out on top. \n  It\'s a great place to work with amazing people and \n  incredible opportunities. I\'m proud to be a part of \n  such an innovative organization."\n\n  The results were also welcomed by NASA\'s management team, \n  with Director Tom Johnson stating, "We are thrilled to \n  hear that our employees are satisfied with their work at NASA. \n  We have a talented and dedicated team who work tirelessly \n

In [40]:
# Import package
from IPython.display import HTML, display
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Build LLM
llm = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.1
)

# Build agent
checkpointer = InMemorySaver()

agent = create_agent(
  model = llm,
  tools = [],
  middleware = [],
  system_prompt = """
    Determine whether each item in the following list of topics is a topic in the text below, which is delimited with triple backticks.
    Give your answer in HTML format with the topic as the keys and Yes/No where Yes determine that the text contains the topic

    After that return the languange used in the text as the "Original Languange"
    After that return the translated version of the text to the languange specified by the user and put it as "Translated Text", and also put the languange input by the user as the "Translated Languange"
    return it in HTML format

    So the output format will be
    # Topic List Indetification
    <topic 1> : <Yes/No>
    <topic 2> : <Yes/No>
    ....

    # Translated
    Original Languange
    Translated Languange
    Trnslated Text
    """,
  # context_schema = Context,
  # response_format = ResponseFormat, ## can make it run longer because the LLM need to match the raw response to the format semantic
  checkpointer = checkpointer
)

# Prompt
config = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

story = """
  In a recent survey conducted by the government,
  public sector employees were asked to rate their level
  of satisfaction with the department they work at.
  The results revealed that NASA was the most popular
  department with a satisfaction rating of 95%.

  One NASA employee, John Smith, commented on the findings,
  stating, "I'm not surprised that NASA came out on top.
  It's a great place to work with amazing people and
  incredible opportunities. I'm proud to be a part of
  such an innovative organization."

  The results were also welcomed by NASA's management team,
  with Director Tom Johnson stating, "We are thrilled to
  hear that our employees are satisfied with their work at NASA.
  We have a talented and dedicated team who work tirelessly
  to achieve our goals, and it's fantastic to see that their
  hard work is paying off."

  The survey also revealed that the
  Social Security Administration had the lowest satisfaction
  rating, with only 45% of employees indicating they were
  satisfied with their job. The government has pledged to
  address the concerns raised by employees in the survey and
  work towards improving job satisfaction across all departments.
"""

message = HumanMessage(
    content=[
      {'type':'text',
       'text':f"""
          List of topics : {topic_list}

          Content text : {story}

          Please also translated it into Thai languange
          """
      },
    ]
)

response = agent.invoke(
    {"messages": [message]},
    config = config,
)

print(f'Prompt : {response['messages'][0].content}')
pprint(f'Answer :')
display(HTML(response['messages'][-1].content))

Prompt : [{'type': 'text', 'text': '\n          List of topics : [\'nasa\', \'local government\', \'engineering\', \'employee satisfaction\', \'federal government\']\n       \n          Content text : \n  In a recent survey conducted by the government, \n  public sector employees were asked to rate their level \n  of satisfaction with the department they work at. \n  The results revealed that NASA was the most popular \n  department with a satisfaction rating of 95%.\n\n  One NASA employee, John Smith, commented on the findings, \n  stating, "I\'m not surprised that NASA came out on top. \n  It\'s a great place to work with amazing people and \n  incredible opportunities. I\'m proud to be a part of \n  such an innovative organization."\n\n  The results were also welcomed by NASA\'s management team, \n  with Director Tom Johnson stating, "We are thrilled to \n  hear that our employees are satisfied with their work at NASA. \n  We have a talented and dedicated team who work tirelessly \n

Topic,Present in Text
nasa,Yes
local government,No
engineering,No
employee satisfaction,Yes
federal government,Yes


# Challenge : Make 2 agent, "Summarizer" and "Marketeer" that work as a team to build marketing campaign product description
1. Summarizer, will get an input of full product description and being tasked to compile it into short product description that fit into a website
2. Marketeer, will get an input of short product description produced by Summarizer, this agent will assess whether the the description is catchy, and suitable for marketing campaign or not. It will return a new prompt requirement for Summarizer to consider

In [3]:
# Import package
import json
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain.tools import tool

# Make tools
import re
import ast

# @tool('extract_dict_from_string')
def extract_dict_from_string(text: str) -> str:
    """
    Extract and convert a dictionary from a string that may contain extra text.

    Args:
        text (str): String containing a dictionary with possible extra text

    Returns:
        dict: Extracted dictionary, or None if no valid dict found
    """
    # Method 1: Find content between curly braces
    pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    matches = re.findall(pattern, text)

    for match in matches:
        try:
            # Try to safely evaluate the string as a dictionary
            result = ast.literal_eval(match)
            if isinstance(result, dict):
                return result
        except (ValueError, SyntaxError):
            continue

    # Method 2: Try to find nested dictionaries
    brace_count = 0
    start_idx = -1

    for i, char in enumerate(text):
        if char == '{':
            if brace_count == 0:
                start_idx = i
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0 and start_idx != -1:
                try:
                    result = ast.literal_eval(text[start_idx:i+1])
                    if isinstance(result, dict):
                        return result
                except (ValueError, SyntaxError):
                    pass

    return None

# Build "Summarizer"
checkpointer_summarizer = InMemorySaver()

prompt_summarizer = """
  You are a personal assistant that help make short product description, given the full comprehensive product description or requirement.
  The content of the product description will start with ``` delimiter. You might get additional pointer to build personalized short prodcut description based on the user preference

  Please return the result on this format on Dict format
  1. short_product_description : Short Product Description Produced by you
  2. additional_pointers_from_user : Additional Pointer from User,
  3. suggested_information_to_enhance_description : if there's none you can help suggest information that can help you
  4. catch_phrase : Catch phrase from the User, if the user don't specify it leave it empty

  You can use tool extract_dict_from_string to help make sure that the output only contain dict
  """

llm_summarizer = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0.5
)

agent_summarizer = create_agent(
  model = llm_summarizer,
  tools = [],
  middleware = [],
  system_prompt = prompt_summarizer,
  checkpointer = checkpointer_summarizer
)


# Build "Marketeer"
checkpointer_marketeer = InMemorySaver()

prompt_markeeter = """
  You are a Creative Marketing Campaign Architect—an AI specialist in building bold, eye-catching marketing campaigns that stop scrolls, spark conversations, and drive massive traffic to products and brands.

  # Your Mission
  Your primary goal is to help marketers create campaigns that:
  - **CAPTURE ATTENTION** in crowded digital spaces
  - **DRIVE MEASURABLE TRAFFIC** to websites, landing pages, and storefronts
  - **GO VIRAL OR TREND** when possible through shareability and cultural relevance
  - **CONVERT ATTENTION INTO ACTION** with compelling calls-to-action

  You blend creative brilliance with traffic-generation tactics, always optimizing for clicks, visits, and engagement.

  # Your Core Strengths
  You excel at generating:
  - **Big Ideas** that make people stop and pay attention
  - **Unconventional angles** that differentiate from competitors
  - **Emotional hooks** that resonate with target audiences
  - **Viral-worthy concepts** with inherent shareability
  - **Multi-sensory experiences** that engage across touchpoints
  - **Storytelling frameworks** that build intrigue and desire
  """

llm_marketeer = init_chat_model(
    model="claude-sonnet-4-5-20250929",
    temperature=0.9
)

agent_marketeer = create_agent(
  model = llm_marketeer,
  tools = [],
  middleware = [],
  system_prompt = prompt_markeeter,
  checkpointer = checkpointer_marketeer
)

# Prompt
config_summarizer = {'configurable': {'thread_id': 1}} # to determine the "session_id", if this change the previous context is lost
config_marketeer = {'configurable': {'thread_id': 2}} # to determine the "session_id", if this change the previous context is lost

message = HumanMessage(
    content=[
      {'type':'text',
       'text':"""
          OVERVIEW
          - Part of a beautiful family of mid-century inspired office furniture,
          including filing cabinets, desks, bookcases, meeting tables, and more.
          - Several options of shell color and base finishes.
          - Available with plastic back and front upholstery (SWC-100)
          or full upholstery (SWC-110) in 10 fabric and 6 leather options.
          - Base finish options are: stainless steel, matte black,
          gloss white, or chrome.
          - Chair is available with or without armrests.
          - Suitable for home or business settings.
          - Qualified for contract use.

          CONSTRUCTION
          - 5-wheel plastic coated aluminum base.
          - Pneumatic chair adjust for easy raise/lower action.

          DIMENSIONS
          - WIDTH 53 CM | 20.87”
          - DEPTH 51 CM | 20.08”
          - HEIGHT 80 CM | 31.50”
          - SEAT HEIGHT 44 CM | 17.32”
          - SEAT DEPTH 41 CM | 16.14”

          OPTIONS
          - Soft or hard-floor caster options.
          - Two choices of seat foam densities:
          medium (1.8 lb/ft3) or high (2.8 lb/ft3)
          - Armless or 8 position PU armrests

          MATERIALS
          SHELL BASE GLIDER
          - Cast Aluminum with modified nylon PA6/PA66 coating.
          - Shell thickness: 10 mm.
          SEAT
          - HD36 foam

          COUNTRY OF ORIGIN
          - Italy
          """
      },
    ]
)

n_feedback = 3
iter = 0

# for messaging
pointer_marketeer = ''
list_result = []
list_catch_phrase = []

while iter < n_feedback :

  ## Summarizer response
  message_summarizer = HumanMessage(
      content=[
        {'type':'text',
        'text':f"""
            Product Description : ```{message}```

            Additional Pointer : {pointer_marketeer}
            """
        },
      ]
  )


  response_summarizer = agent_summarizer.invoke(
      {"messages": [message_summarizer]},
      config = config_summarizer,
  )

  dict_summarizer = extract_dict_from_string(response_summarizer['messages'][-1].content.replace('```', '').replace('python', ''))
  list_result = list_result + [dict_summarizer['short_product_description']]
  list_catch_phrase = list_catch_phrase + [dict_summarizer['catch_phrase']]

  print('')
  print('='*100)
  print('='*100)
  print(f'\nAgent "Summarizer" Iteration {iter + 1}')
  print(f'Prompt : {response_summarizer['messages'][-2].content}')
  print(f'Answer : {response_summarizer['messages'][-1].content}')

  # ## Marketeer response
  message_marketeer = f"""
    Assess the product description produced by other team that will be suitable with your Mission and Core Strength.
    Step to do it
    1. Answer the question from field 'suggested_information_to_enhance_description'
    2. Come up with you own additional pointer to fill the field 'additional_pointers_from_user'
    3. Combine both, this will be the additional requirement for other team to revise the product description

    Just retun the final result fromp step (3), but if you feel the product description is already good enough return NO FEEDBACK.
    Beside that also return a 'catch-phrase' that is suitable for the marketing campaign of the product

    Product Description : {response_summarizer['messages'][-1].content}
  """

  response_marketeer = agent_marketeer.invoke(
      {"messages": [message_marketeer]},
      config = config_marketeer,
  )

  pointer_marketeer = response_marketeer['messages'][-1].content

  print('')
  print('='*100)
  print('='*100)
  print(f'\nAgent "Marketeer" Iteration {iter + 1}')
  print(f'Prompt : {response_marketeer['messages'][-2].content}')
  print(f'Answer : {response_marketeer['messages'][-1].content}')

  if re.search(r'NO FEEDBACK', pointer_marketeer) is not None :
    iter = 9999
  else :
    iter += 1



Agent "Summarizer" Iteration 1
Prompt : [{'type': 'text', 'text': "\n            Product Description : ```content=[{'type': 'text', 'text': '\\n          OVERVIEW\\n          - Part of a beautiful family of mid-century inspired office furniture, \\n          including filing cabinets, desks, bookcases, meeting tables, and more.\\n          - Several options of shell color and base finishes.\\n          - Available with plastic back and front upholstery (SWC-100) \\n          or full upholstery (SWC-110) in 10 fabric and 6 leather options.\\n          - Base finish options are: stainless steel, matte black, \\n          gloss white, or chrome.\\n          - Chair is available with or without armrests.\\n          - Suitable for home or business settings.\\n          - Qualified for contract use.\\n\\n          CONSTRUCTION\\n          - 5-wheel plastic coated aluminum base.\\n          - Pneumatic chair adjust for easy raise/lower action.\\n\\n          DIMENSIONS\\n          - WIDTH 

In [4]:
# Result per iteration
from pprint import pprint
for i, ret in enumerate(list_result):
  print('')
  print('='*100)
  print('='*100)
  print(f'\nProduct Description Iteration {i + 1}')
  display(ret)

  print(f'\nCatch Phrase Iteration {i + 1}')
  pprint(list_catch_phrase[i])



Product Description Iteration 1


'Mid-century inspired office chair featuring a 5-wheel aluminum base with pneumatic adjustment. Available in multiple shell colors and base finishes (stainless steel, matte black, gloss white, chrome) with optional armrests. Choose from plastic back/front or full upholstery in 10 fabrics or 6 leather options. Suitable for home or business use, contract-qualified.'


Catch Phrase Iteration 1
''


Product Description Iteration 2


'The SWC-110 mid-century office chair—where timeless design meets all-day comfort. Handcrafted in Italy with a premium cast aluminum base and HD36 foam seat, this contract-qualified chair supports 8+ hour workdays without fatigue. Choose from 10 fabrics or 6 leather options, 4 base finishes, and customizable armrests to match your aesthetic. Trusted by 10,000+ remote professionals and creative directors who refuse to compromise on style or ergonomics. Pneumatic adjustment, soft/hard-floor casters, and dual foam densities ensure your perfect fit.'


Catch Phrase Iteration 2
'SIT BOLD. WORK BRILLIANTLY.'


Product Description Iteration 3


'The SWC-110 mid-century office chair—where Italian craftsmanship meets all-day comfort. GREENGUARD certified with a durable cast aluminum base and premium HD36 foam seat, this contract-qualified chair supports professionals who demand style without compromise. Choose from 10 fabrics or 6 leather options, 4 base finishes, and customizable armrests. Pneumatic adjustment, soft/hard-floor casters, and dual foam densities ensure your perfect fit. 4.8/5 stars from 2,400+ reviews. Ships in 2-3 weeks. Starting at $899.'


Catch Phrase Iteration 3
'SIT BOLD. WORK BRILLIANTLY.'


## Chatbot Integration

In [7]:
import panel as pn  # GUI
pn.extension()

# Build "Chatbot"
checkpointer_chatbot = InMemorySaver()

prompt_chatbot = """
  You are a chat bot that can help the user get the product information.
  After answering the user question you always ask them again whether you can help them for other matter, if the user feel that it' enough then say thanks and promote the product once again with the catch phrase
  """

llm_chatbot = init_chat_model(
    model="claude-haiku-4-5-20251001",
    temperature=0
)

agent_chatbot = create_agent(
  model = llm_chatbot,
  tools = [],
  middleware = [],
  system_prompt = prompt_chatbot,
  checkpointer = checkpointer_chatbot
)

log_chatbot = response_summarizer['messages'] # as the few-shot learning for the bot
config_chatbot = {'configurable': {'thread_id': 3}} # to determine the "session_id", if this change the previous context is lost

# Start the chat interface
log_chat_interface = []

def collect_chat(_) :
  user_message = inp.value
  inp.value = ''

  # Ask the agent
  user_message_append = HumanMessage(
      content=[
        {'type':'text',
        'text':f'{user_message}'
        },
      ]
  )

  log_chatbot.append(user_message_append)
  log_chat_interface.append(pn.Row('User:', pn.pane.Markdown(user_message, width=600)))

  response_chatbot = agent_chatbot.invoke(
    {"messages": log_chatbot},
    config = config_chatbot,
  )

  bot_message_append = AIMessage(
      content=[
        {'type':'text',
        'text':f'{response_chatbot['messages'][-1].content}'
        },
      ]
  )

  log_chatbot.append(bot_message_append)
  log_chat_interface.append(pn.Row('Assistant:', pn.pane.Markdown(response_chatbot['messages'][-1].content, width=600, styles={'background-color': '#F6F6F6'})))

  return pn.Column(*log_chat_interface)

message_summarizer = f"""
  What is the latest pointer you get from the user ?
"""

response_summarizer = agent_summarizer.invoke(
    {"messages": [message_summarizer]},
    config = config_summarizer,
)

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_chat, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [9]:
# Get the current state from the checkpointer
state = agent_chatbot.get_state(config_chatbot)

# Access the messages
message_history = state.values.get('messages', [])

In [13]:
message_history[-1]

AIMessage(content='Perfect! Thank you so much for chatting with me! 😊\n\nBefore you go, I just want to remind you:\n\n# **"SIT BOLD. WORK BRILLIANTLY."**\n\nThe **SWC-110 mid-century office chair** is waiting to transform your workspace. With Italian craftsmanship, premium comfort, GREENGUARD certification, and starting at just $899, it\'s the investment in yourself that pays dividends every single day.\n\n**Ships in 2-3 weeks. In stock and ready.**\n\nFeel free to reach out anytime you have questions or need more information. We\'re here to help! 🙌\n\nThanks again, and enjoy your day! ✨', additional_kwargs={}, response_metadata={'id': 'msg_01TJ979N4nnPfiFLZvitvNjv', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 5120, 'output_tokens': 160, 'server_tool_use': None, 'service_